<a href="https://colab.research.google.com/github/samarzeit/Physics_Sandbox/blob/main/notebook/Creating%20folders%20Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import torch.utils.data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F


import os, shutil
from PIL import Image
import re
import random
import numpy as np
import matplotlib.pyplot as plt
import torchvision


from tqdm import tqdm


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # use GPU if available

import os, shutil, random

In [29]:
print(device)

cuda


In [30]:
!git clone https://github.com/vc1492a/Hey-Waldo.git


fatal: destination path 'Hey-Waldo' already exists and is not an empty directory.


# Creating initial 4 folders

In [31]:
#https://www.freecodecamp.org/news/creating-a-directory-in-python-how-to-create-a-folder/



for path in ['Positive', 'Anchor' , 'Negative', 'IntermediateFolder']:
    if not os.path.exists(path):
      os.mkdir(path)
      print("Folder %s created!" % path)
    else:
      print("Folder %s already exists" % path)


Folder Positive already exists
Folder Anchor already exists
Folder Negative already exists
Folder IntermediateFolder already exists


# Required functions

In [32]:

def copy_all_files(source,destination):
    for file in os.listdir(source):
        full_file_path = os.path.join(source, file)
        destination_file_path = os.path.join(destination,file)

        if os.path.exists(full_file_path):
            shutil.copyfile(full_file_path, destination_file_path)
        else:
            print("Skipping file:", full_file_path, "(file does not exist)")



def move_all_files (source, destination):
    for file in os.listdir(source):
        full_file_path = os.path.join(source, file)
        destination_file_path = os.path.join(destination, file)

        if os.path.exists(full_file_path):
            shutil.move(full_file_path, destination_file_path)
        else:
            print("Skipping file:", full_file_path, "(file does not exist)")

def empty_folder(folder):
    for file in os.listdir(folder):
        full_file_path = os.path.join(folder, file)

        if os.path.exists(full_file_path):
            os.remove(full_file_path)
        else:
            print(f"Skipping file: {full_file_path}, file does not exist")


# def add_label(folder,label):
#     for file in os.listdir(folder):
#         f_name, f_ext = os.path.splitext(file)
#         f_name = f_name +'_label'+ str(label)
#         new_name = f'{f_name}{f_ext}'
#         os.rename(file, new_name)





def add_label(folder,label):
    for file in os.listdir(folder):
        full_file_path = os.path.join(folder, file)

        if os.path.exists(full_file_path):
            f_name, f_ext = os.path.splitext(file)

            # new_file_name = f_name +'_label' + str(label)+ f_ext
            new_file_name = 'label' + str(label)+ '_' + f_name + f_ext
            new_file_path = os.path.join(folder, new_file_name)

            if not os.path.exists(new_file_path):
                os.rename(full_file_path, new_file_path)
            else:
                print(f"Skipping file {full_file_path}. File with matching name and label already exists.")
        else:
            print(f"Skipping file: {full_file_path}, file does not exist")



def move_file(source_file, destination_path):
    if os.path.exists(source_file) and os.path.exists(destination_path):
         # Extract filename from source path
        filename = os.path.basename(source_file)

            # Join destination path and filename to create the new path
        destination_file = os.path.join(destination_path, filename)

            # Move the file
        shutil.move(source_file, destination_file)

        print(f"File '{filename}' moved to '{destination_path}' successfully!")

    else:
        print("Source file or destination path does not exist.")


# Loading data to the 3 folders

In [33]:
# empty_folder('Positive')
# empty_folder('Anchor')
# empty_folder('Negative')
# empty_folder('IntermediateFolder')


# empty_folder('Test_Positive')
# empty_folder('Test_Anchor')
# empty_folder('Test_Negative')


In [34]:
# empty_folder('IntermediateFolder')
copy_all_files('Hey-Waldo/128/waldo','IntermediateFolder')
add_label('IntermediateFolder',1)
copy_all_files('IntermediateFolder','Positive')
copy_all_files('IntermediateFolder','Anchor')
empty_folder('IntermediateFolder')


In [35]:
# empty_folder('IntermediateFolder')
copy_all_files('Hey-Waldo/128/notwaldo','IntermediateFolder')
add_label('IntermediateFolder',0)
copy_all_files('IntermediateFolder','Negative')
copy_all_files('IntermediateFolder','Anchor')
empty_folder('IntermediateFolder')

## After moving 128 bit images

The number of files in the 3 folders after moving the 128 bit images

In [36]:
# print(os.getcwd())
print(len(os.listdir('Anchor')))
print('----------')
print(len(os.listdir('Positive')))
print(len(os.listdir('Negative')))

4601
----------
60
4547


In [37]:
empty_folder('IntermediateFolder')
copy_all_files('Hey-Waldo/64/waldo','IntermediateFolder')
add_label('IntermediateFolder',1)
copy_all_files('IntermediateFolder','Positive')
copy_all_files('IntermediateFolder','Anchor')
empty_folder('IntermediateFolder')

In [38]:
empty_folder('IntermediateFolder')
copy_all_files('Hey-Waldo/64/notwaldo','IntermediateFolder')
add_label('IntermediateFolder',0)
copy_all_files('IntermediateFolder','Negative')
copy_all_files('IntermediateFolder','Anchor')
empty_folder('IntermediateFolder')

## After moving 64 bit images

The number of files in the 3 folders after moving the 64 bit images

In [39]:
# print(os.getcwd())
print(len(os.listdir('Anchor')))
print('----------')
print(len(os.listdir('Positive')))
print(len(os.listdir('Negative')))

5419
----------
66
5353


# Creation of additional 3 folders for test dataset

Some files are moved from the above folders to create the test dataset


In [40]:

# Define paths for original folders and test folders
positive_folder = 'Positive'
negative_folder = 'Negative'
anchor_folder = 'Anchor'
test_positive_folder = 'Test_Positive'
test_negative_folder = 'Test_Negative'
test_anchor_folder = 'Test_Anchor'

# Randomly select files (adjust the percentage as needed)
percentage_test_data = 0.2  # 20% of data for testing
positive_files = os.listdir(positive_folder)
negative_files = os.listdir(negative_folder)
anchor_files = os.listdir(anchor_folder)

num_positive_test = int(len(positive_files) * percentage_test_data)
num_negative_test = int(len(negative_files) * percentage_test_data)
num_anchor_test = int(len(anchor_files) * percentage_test_data)

random.seed(42)  # For reproducibility
selected_positive_files = random.sample(positive_files, num_positive_test)
selected_negative_files = random.sample(negative_files, num_negative_test)
selected_anchor_files = random.sample(anchor_files, num_anchor_test)

# Create test folders if they don't exist
os.makedirs(test_positive_folder, exist_ok=True)
os.makedirs(test_negative_folder, exist_ok=True)
os.makedirs(test_anchor_folder, exist_ok=True)


# Move or copy selected files to test folders
for file in selected_positive_files:
    move_file(f'/content/Positive/{file}', '/content/Test_Positive')

for file in selected_negative_files:
    move_file(f'/content/Negative/{file}', '/content/Test_Negative')

for file in selected_anchor_files:
    move_file(f'/content/Anchor/{file}', '/content/Test_Anchor')

File 'label1_12_2_1.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_9_0_6.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_6_15_1.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_3_15_1.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_4_1_5.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_9_2_2.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_9_9_5.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_4_2_12.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_13_2_1.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_17_1_5.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_1_4_6.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_9_0_5.jpg' moved to '/content/Test_Positive' successfully!
File 'label1_6_7_0.jpg' moved to '/content/Test_Positive' successfully!
File 'label0_17_1_5.jpg' moved to '/content/Test_Negative'

## Size of all 6 folders

Here is the size of all 6 folders after moving few files to test dataset folders

In [41]:
# print(os.getcwd())
print(len(os.listdir('Anchor')))
print('----------')
print(len(os.listdir('Positive')))
print(len(os.listdir('Negative')))

print('*******************************')

# print(os.getcwd())
print(len(os.listdir('Test_Anchor')))
print('----------')
print(len(os.listdir('Test_Positive')))
print(len(os.listdir('Test_Negative')))


4336
----------
53
4283
*******************************
1083
----------
13
1070


# rough work


In [42]:
os.getcwd()


'/content'

In [43]:
def extract_label(filename):
    regex = r'label(\d)'  # Matches the string 'label' followed by a single digit

    match = re.findall(regex, filename)
    if match:
        return match[0]
    else:
        return None

In [44]:
class TripletDataset(Dataset):
    def __init__(self,waldo_folder, not_waldo_folder, anchor_folder,transform=None):

        self.waldo_folder=waldo_folder
        self.not_waldo_folder=not_waldo_folder
        self.anchor_folder=anchor_folder

        self.waldo_images = os.listdir(waldo_folder)  # List of Waldo images (file paths or PIL images)
        self.not_waldo_images = os.listdir(not_waldo_folder)  # List of non-Waldo images (file paths or PIL images)
        self.anchor_images= os.listdir(anchor_folder)
        self.transform = transform



        self.waldo_images_count =  len(self.waldo_images)
        self.not_waldo_images_count =  len(self.not_waldo_images)

    def __getitem__(self, index):
        anchor_file_name=self.anchor_images[index]
        anchor_label= extract_label(anchor_file_name)
        file1=os.path.join(self.anchor_folder, anchor_file_name)
        anchor = Image.open(file1)

        if anchor_label==0: #not waldo case
            random_ind=random.randint(0,self.not_waldo_images_count-1)
            positive_file= self.not_waldo_images[random_ind]
            file2=os.path.join(self.not_waldo_folder, positive_file)
            positive = Image.open(file2)

            random_ind=random.randint(0,self.waldo_images_count-1)
            negative_file=self.waldo_images[random_ind]
            file3=os.path.join(self.waldo_folder, negative_file)
            negative = Image.open(file3)

        else: # waldo case ====> anchor_label==1
            random_ind=random.randint(0,self.waldo_images_count-1)
            positive_file= self.waldo_images[random_ind]
            file4=os.path.join(self.waldo_folder, positive_file)
            negative = Image.open(file4)

            random_ind=random.randint(0,self.not_waldo_images_count-1)
            negative_file=self.not_waldo_images[random_ind]
            file5=os.path.join(self.not_waldo_folder, negative_file)
            positive = Image.open(file5)



        if self.transform:
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive, negative


    def __len__(self):
        return len(self.anchor_images)  # You can choose any length based on your dataset design



# class TripletDataset(Dataset):
#     def __init__(self, waldo_folder, not_waldo_folder, anchor_folder, transform=None):
#         self.waldo_folder=waldo_folder
#         self.not_waldo_folder=not_waldo_folder
#         self.anchor_folder=anchor_folder

#         self.waldo_images = os.listdir(waldo_folder)
#         self.not_waldo_images = os.listdir(not_waldo_folder)
#         self.anchor_images = os.listdir(anchor_folder)
#         self.transform = transform

#         self.waldo_images_count = len(self.waldo_images)
#         self.not_waldo_images_count = len(self.not_waldo_images)

#     def __getitem__(self, index):
#         anchor_file_name = self.anchor_images[index]
#         anchor_label = extract_label(anchor_file_name)
#         file1 = os.path.join(self.anchor_folder, anchor_file_name)
#         anchor = Image.open(file1)

#         if anchor_label == 0:  # not waldo case
#             random_ind = random.randint(0, self.not_waldo_images_count - 1)
#             negative_file_name = self.not_waldo_images[random_ind]
#             file2 = os.path.join(self.not_waldo_folder, negative_file_name)
#             negative = Image.open(file2)

#             random_ind = random.randint(0, self.waldo_images_count - 1)
#             positive_file_name = self.waldo_images[random_ind]
#             file3 = os.path.join( self.waldo_folder, positive_file_name)
#             positive = Image.open(file3)
#         else:  # waldo case
#             random_ind = random.randint(0, self.waldo_images_count - 1)
#             positive_file_name = self.waldo_images[random_ind]
#             file4 = os.path.join(self.waldo_folder, positive_file_name)
#             positive = Image.open(file4)

#             random_ind = random.randint(0, self.not_waldo_images_count - 1)
#             negative_file_name = self.not_waldo_images[random_ind]
#             file5 = os.path.join(self.not_waldo_folder, negative_file_name)
#             negative = Image.open(file5)

#         if self.transform:
#             anchor = self.transform(anchor)
#             positive = self.transform(positive)
#             negative = self.transform(negative)

#         return anchor, positive, negative

#     def __len__(self):
#         return len(self.anchor_images)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    # Add other transformations as needed
])

In [45]:
batch_size = 32

# Set up data loaders
triplet_dataset = TripletDataset('Positive', 'Negative', 'Anchor',transform=transform)
train_loader = DataLoader(triplet_dataset, batch_size=batch_size, shuffle=True)

# Create test dataset instance
test_triplet_dataset = TripletDataset('Test_Positive', 'Test_Negative', 'Test_Anchor', transform=transform)
test_loader = DataLoader(test_triplet_dataset, batch_size=batch_size, shuffle=True)




In [ ]:
for batch_idx, (anchor, positive, negative) in enumerate(train_loader):
    # Print the shape of each batch to verify the data loading
    print(f"Batch {batch_idx}:")
    print(f"Anchor shape: {anchor.shape}")
    print(f"Positive shape: {positive.shape}")
    print(f"Negative shape: {negative.shape}")

In [ ]:
# Assuming you have set up your train_loader
for batch_idx, (anchor, positive, negative) in enumerate(test_loader):
    # Print the shape of each batch to verify the data loading
    print(f"Batch {batch_idx}:")
    print(f"Anchor shape: {anchor.shape}")
    print(f"Positive shape: {positive.shape}")
    print(f"Negative shape: {negative.shape}")
    # Add further inspection or analysis as needed
    # Ensure your data shapes are as expected for further processing


In [ ]:
print(train_loader.batch_size)
print(test_loader.batch_size)

32
32


In [47]:
class Patchify(nn.Module):

    def __init__(self, img_size, patch_size, embed_dim):
        super().__init__()
        self.num_patches = (img_size // patch_size) * (img_size // patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_channels = 3,
                              out_channels = embed_dim,
                              kernel_size = patch_size,
                              stride = patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)   # (batch_size, dim, img_size // patch_size, img_size // patch_size)
        # the input to the transformer should be of shape (batch_size, num_patches, embedding dim)
         # (batch_size, dim, img_size // patch_size, img_size // patch_size) --> flattten --> (batch_size, dim, num_patches) --> tranpose --> (batch_size, num_patches, dim)
        x = x.flatten(2).transpose(1,2)
        return x

In [ ]:
# from WPO
class FeedForward(nn.Module):
    def __init__(self, dim, dropout = 0.):
        super().__init__()

        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class MultiHeadAttention(nn.Module):

    """Creates a multi-head self-attention block ("MSA block" for short).
    """
    # 2. Initialize the class with hyperparameters from Table 1
    def __init__(self,
                 embedding_dim, # Hidden size D from Table 1 for ViT-Base
                 heads, # Heads from Table 1 for ViT-Base
                 dropout=0): # doesn't look like the paper uses any dropout in MSABlocks
        super().__init__()

        # 3. Create the Norm layer (LN)
        self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

        # 4. Create the Multi-Head Attention (MSA) layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                    num_heads=heads,
                                                    dropout=dropout,
                                                    batch_first=True) # does our batch dimension come first?

    # 5. Create a forward() method to pass the data throguh the layers
    def forward(self, x):
        x = self.layer_norm(x)
        attn_output, _ = self.multihead_attn(query=x, # query embeddings
                                             key=x, # key embeddings
                                             value=x, # value embeddings
                                             need_weights=False) # do we need the weights or just the layer outputs?
        return attn_output
    # def __init__(self, dim, heads = 8, dropout = 0.):
    #     super().__init__()
    #     self.dim = dim
    #     self.heads = heads

    #     self.query = nn.Linear(dim, dim)
    #     self.keys = nn.Linear(dim, dim)
    #     self.values = nn.Linear(dim, dim)

    #     self.norm = nn.LayerNorm(dim)

    #     self.out_proj = nn.Linear(dim, dim)

    #     self.dropout = nn.Dropout(dropout)

    # def forward(self, x):
    #     # N is the total number of patches
    #     # print('*agas*fga*sdg*s*gas*g*dsf*gsdfg*dsf*gsd*fg*sdfg*')
    #     # print(x.shape)
    #     # print('*agas*fga*sdg*s*gas*g*dsf*gsdfg*dsf*gsd*fg*sdfg*')
    #     # x = x.flatten(2).transpose(1, 2)

    #     B, N, C = x.shape
    #     # B=32
    #     # N=1

    #     x = self.norm(x)         # (B, N, C)

    #     query = self.query(x)    # (B, N, C)
    #     key = self.keys(x)       # (B, N, C)
    #     value = self.values(x)   # (B, N, C)

    #     dim_head = C // self.heads

    #     # Split (B,N,C) into (B, N, num_heads, dim_head) and permute heads which yields a shape of (B, num_heads, N, dim_head)
    #     # each of the heads, should have (N, dim_head)
    #     query = query.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)
    #     key = key.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)
    #     value = value.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)

    #     # (B, num_heads, N, dim_head) with (B, num_heads, N, dim_head) --> (B, num_heads, N, N)
    #     attention_scores = torch.matmul(query, key.transpose(-1, -2))

    #     scale = dim_head ** -0.5   # (1 / sqrt(dim_head))
    #     attention_scores = attention_scores * scale
    #     attention_scores = F.softmax(attention_scores, dim = -1) # (B, num_heads, N, N)
    #     attention_scores = self.dropout(attention_scores)

    #     # extract the values
    #     # (B, num_heads, N, N) matmul (B, num_heads, N, dim_head) --> (B, num_heads, N, dim_head)
    #     out = torch.matmul(attention_scores, value)

    #     # (B, num_heads, N, dim_head) --> (B, N, num_heads, dim_head) --> (B, N, C)
    #     out = out.permute(0,2,1,3).flatten(2)   # or we can use .reshape(B, N, -1) rather than .flatten(2)
    #     out = self.out_proj(out)
    #     return out

In [ ]:
class Transformer(nn.Module):
    def __init__(self, dim, num_layers, heads, dropout = 0.):

        super().__init__()

        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])

        for _ in range(num_layers):
            self.layers.append(nn.ModuleList([
                MultiHeadAttention(dim, heads = heads, dropout = dropout),
                FeedForward(dim, dropout = dropout)]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

In [48]:
# class MyViT(nn.Module):
#     def __init__(self,dim,num_layers,heads,dropout):
#         super().__init__()
#         self.patchify=Patchify(128, 128, dim)
#         self.transformer= Transformer(dim, num_layers, heads, dropout = dropout)


#     def forward(self,x):
#         x = self.patchify(x)
#         x=self.transformer(x)
#         x=x.mean(1)
#         return x

class MyViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, num_layers, heads, dropout):
        super().__init__()

        self.patchify = Patchify(image_size, patch_size, dim)
        # 5. Create learnable class embedding (needs to go at front of sequence of patch embeddings)
        self.class_embedding = nn.Parameter(data=torch.randn(1, 1, dim))
        self.pos_encoding = nn.Parameter(torch.randn(1, self.patchify.num_patches+1, dim))
        self.transformer = Transformer(dim, num_layers, heads, dropout = dropout)
        self.classifier = nn.Linear(dim, num_classes)

    def forward(self, x):

        # 12. Get batch size
        batch_size = x.shape[0]

        # 13. Create class token embedding and expand it to match the batch size (equation 1)
        class_token = self.class_embedding.expand(batch_size, -1, -1)
        # 14. Create patch embedding (equation 1)
        # x = self.patch_embedding(x)

        # 15. Concat class embedding and patch embedding (equation 1)

        x = self.patchify(x)         # (batch_size, N, dim)

        x = torch.cat((class_token, x), dim=1)

        x = x + self.pos_encoding    # (batch_size, N, dim)
        x = self.transformer(x)      # (batch_size, N, dim)
        x = x.mean(1)                # (batch_size, dim)
        # x = self.classifier(x)
        return x

In [49]:
# this cell is for checking our model and yes, this transformer is working and giving the output in required shape

model = MyViT(image_size = 128,
            patch_size = 16,
            num_classes = 2,
            dim = 512,
            num_layers = 6,
            heads = 8,
            dropout = 0.1).to(device)

random_input = torch.randn(1,3,128,128).to(device)
print(model(random_input).shape)

torch.Size([1, 512])


In [50]:
# Set up your optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
triplet_loss = nn.TripletMarginLoss(margin=1.0)

# Assuming you have train_loader and test_loader for your dataset
num_epochs = 30

In [ ]:
train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (anchor, positive, negative) in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass to get embeddings
        # anchor = anchor.to(device)
        # positive = positive.to(device)
        # negative = negative.to(device)
        # anchor_out = model(anchor)
        # positive_out = model(positive)
        # negative_out = model(negative)

        anchor_out = model(anchor.to(device))
        positive_out = model(positive.to(device))
        negative_out = model(negative.to(device))


        # Calculate triplet loss
        loss = triplet_loss(anchor_out, positive_out, negative_out)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print batch statistics
        if batch_idx % 100 == 99:  # Print every 100 mini-batches
            print(f"Epoch [{epoch + 1}/{num_epochs}] "
                  f"Batch [{batch_idx + 1}/{len(train_loader)}] "
                  f"Loss: {running_loss / 100:.3f}")
            train_losses.append(running_loss / 100)
            running_loss = 0.0

    # Evaluation on test set after each epoch
    model.eval()

    test_loss = 0.0

    with torch.no_grad():
        for batch_idx, (anchor, positive, negative) in enumerate(test_loader): ##################
            # anchor = anchor.to(device)
            # positive = positive.to(device)
            # negative = negative.to(device)
            # anchor_out = model(anchor)
            # positive_out = model(positive)
            # negative_out = model(negative)

            anchor_out = model(anchor.to(device))
            positive_out = model(positive.to(device))
            negative_out = model(negative.to(device))

            # Calculate cosine similarity between anchor-positive pairs
            similarity_anchor_positive = F.cosine_similarity(anchor_out, positive_out)

            # Calculate cosine similarity between anchor-negative pairs
            similarity_anchor_negative = F.cosine_similarity(anchor_out, negative_out)

            # Calculate the difference between positive and negative similarities
            similarity_difference = similarity_anchor_positive - similarity_anchor_negative


            similarity_positive_negative = F.cosine_similarity(positive_out, negative_out)


            loss = triplet_loss(anchor_out, positive_out, negative_out)
            test_loss += loss.item()
        test_losses.append(test_loss / len(test_loader))
        # Print the similarity scores for a sample
        print(f"Similarity Anchor-Positive:   {similarity_anchor_positive[0].item():.3f}")
        print(f"Similarity Anchor-Negative:  { similarity_anchor_negative[0].item():.3f}")
        print(f"Difference (Pos - Neg):   {similarity_difference[0].item():.3f}")

        print(f"Similarity Positive-Negative:  { similarity_positive_negative[0].item():.3f}")


    print(f"Epoch [{epoch + 1}/{num_epochs}] "
          f"Test Loss: {test_loss / len(test_loader):.3f}")
    print("***********************************")

print("Training finished.")

Epoch [1/30] Batch [100/136] Loss: 0.847
Similarity Anchor-Positive:   1.000
Similarity Anchor-Negative:  0.997
Difference (Pos - Neg):   0.003
Similarity Positive-Negative:  0.998
Epoch [1/30] Test Loss: 0.916
***********************************
Epoch [2/30] Batch [100/136] Loss: 0.478
Similarity Anchor-Positive:   1.000
Similarity Anchor-Negative:  0.936
Difference (Pos - Neg):   0.064
Similarity Positive-Negative:  0.943
Epoch [2/30] Test Loss: 0.912
***********************************
Epoch [3/30] Batch [100/136] Loss: 0.287
Similarity Anchor-Positive:   1.000
Similarity Anchor-Negative:  0.997
Difference (Pos - Neg):   0.003
Similarity Positive-Negative:  0.998
Epoch [3/30] Test Loss: 0.884
***********************************
Epoch [4/30] Batch [100/136] Loss: 0.561
Similarity Anchor-Positive:   1.000
Similarity Anchor-Negative:  0.998
Difference (Pos - Neg):   0.002
Similarity Positive-Negative:  0.999
Epoch [4/30] Test Loss: 0.882
***********************************
Epoch [5/30]

In [ ]:
axes = plt.axes()
# axes.set_ylim(0.0, 0.1)

plt.plot(train_losses)
plt.xlabel('epoch')
plt.ylabel('train loss')

In [ ]:
# axes.set_ylim(0.0, 1.0)

plt.plot(test_losses)
plt.xlabel('epoch')
plt.ylabel('test loss')
